In [3]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

columns=['buying','maint','doors','persons','lug_boot','safety','class']
df = pd.read_csv('D:\car.data',names=columns)
df.head(10)


,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


In [4]:
for c in columns:
    df[c] = pd.factorize(df[c])[0]
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,2,0
3,0,0,0,0,1,0,0
4,0,0,0,0,1,1,0


In [5]:
x = df.drop('class', axis=1)  
y = df['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state = 0)

In [6]:
n_estimators = [3,5,10,50,100]
max_depth = [1,7,10]
min_samples_leaf = [2,5,10]


results = pd.DataFrame(columns=[
    "n_estimators", "max_depth","min_samples_leaf",
    "score (train)","score (test)"
])

for ne in n_estimators:
    for md in max_depth:
        for ml in min_samples_leaf:
            rf = RandomForestClassifier(
                n_estimators=ne,
                min_samples_leaf=ml,
                max_depth=md
            )

            rf.fit(x_train,y_train)
            strain = rf.score(x_train,y_train)
            stest = rf.score(x_test,y_test)

            results.loc[results.shape[0]] = [ne,md,ml,strain,stest]

results = results.sort_values('score (test)', ascending=False)
results.head(10)

,n_estimators,max_depth,min_samples_leaf,score (train),score (test)
42,100.0,10.0,2.0,0.994814,0.952715
33,50.0,10.0,2.0,0.992221,0.950963
24,10.0,10.0,2.0,0.991357,0.942207
15,5.0,10.0,2.0,0.986171,0.942207
34,50.0,10.0,5.0,0.978392,0.940455
30,50.0,7.0,2.0,0.972342,0.938704
39,100.0,7.0,2.0,0.968885,0.935201
26,10.0,10.0,10.0,0.964564,0.933450
25,10.0,10.0,5.0,0.967156,0.933450
43,100.0,10.0,5.0,0.972342,0.933450


In [7]:
rf = RandomForestClassifier(
        n_estimators=int(results.iloc[0]['n_estimators']),
        min_samples_leaf=int(results.iloc[0]['min_samples_leaf']),
        max_depth=int(results.iloc[0]['max_depth'])
    )

rf.fit(x_train,y_train)
print(f'Точность = {rf.score(x_test,y_test):.4f}')

import pickle

with open('rfmodel', 'wb') as file:
    pickle.dump(rf, file)
    print('Сохранено в файл')

with open('rfmodel', 'rb') as file:
    rf = pickle.load(file)
    print('Загружено из файла')
    
print(f'Точность = {rf.score(x_test,y_test):.4f}\n\n')


feat_importances = pd.Series(rf.feature_importances_, index=x.columns).sort_values(ascending=False)
print(feat_importances)

Точность = 0.9562
Сохранено в файл
Загружено из файла
Точность = 0.9562


safety      0.317939
persons     0.273983
buying      0.154359
maint       0.128450
lug_boot    0.078718
doors       0.046552
dtype: float64
